In [1]:
import logging
import cobrakbase
import sys
sys.path.append('../lib')

import mightymorphingmodels
from mightymorphingmodels.GrowthConditions import CobraCondition
from mightymorphingmodels.morph import Morph
from mightymorphingmodels.objects import *

logger = logging.getLogger(__name__)

cobrakbase 0.2.2


In [2]:
kbase = cobrakbase.KBaseAPI('<TOKEN>')

In [3]:
ws = "bking:narrative_1571376470358"

In [4]:
media_id = "2016_01_19_h2co2only_media"
genome_id = "Methanocaldococcus_jannaschii_DSM_2661"
fbamodel_id = "MaripaludisModelRevisedCleanIds"
protcomp_id = "maripaludis_to_jannaschii_9_17_19"
rxn_probs_id = "Methanocaldococcus_jannaschii_DSM_2661.rxnprobs"
#to skip steps
trans_model_id = "translated_MaripaludisModelRevisedCleanIds"
recon_model_id = "recon_Methanocaldococcus_jannaschii_DSM_2661"

super_model_id    = "super_model"
super_modelspc_id = "super_modelspc"

output_model_id = "py3test"

In [5]:
genome_data = kbase.get_object(genome_id, ws)
media_data = kbase.get_object(media_id, ws)
fbamodel_data = kbase.get_object(fbamodel_id, ws)
protcomp_data = kbase.get_object(protcomp_id, ws)
rxn_probs_data = kbase.get_object(rxn_probs_id, ws)

trans_model_data = kbase.get_object(trans_model_id, ws)
recon_model_data = kbase.get_object(recon_model_id, ws)

In [6]:
protcomp = ProteomeComparison(protcomp_id, ws, data=protcomp_data, persistent=False)
probanno = ReactionProbabilities(rxn_probs_id, ws, data=rxn_probs_data, persistent=False)
media = Media(media_id, ws, data=media_data, persistent=False)
genome = Genome(genome_id, ws, data=genome_data, persistent=False)
fbamodel = FBAModel(fbamodel_id, ws, data=fbamodel_data, persistent=False)

trans_model = FBAModel(trans_model_id, ws, data=trans_model_data, persistent=False)
recon_model = FBAModel(recon_model_id, ws, data=recon_model_data, persistent=False)

In [7]:
#translated_model = kbase.get_object('translated_MaripaludisModelRevisedCleanIds', ws)
#recon_model = kbase.get_object('recon_Methanosphaera_stadtmanae_DSM_3091', ws)
#super_modelspc = kbase.get_object('super_modelspc', ws)

In [8]:
morph = Morph(service=None,
              src_model=fbamodel,
              media=media,
              probanno=probanno,
              protcomp=protcomp,
              genome=genome,
              ws_id=ws)

In [9]:
#morph.translate_features()
#morph.reconstruct_genome()

morph.trans_model = trans_model
morph.recon_model = recon_model

In [10]:
morph.label_reactions()

In [11]:
super_model_data    = kbase.get_object(super_model_id, ws)
super_modelspc_data = kbase.get_object(super_modelspc_id, ws)
super_model    = FBAModel(super_model_data   , ws, data=super_model_data, persistent=False)
super_modelspc = FBAModel(super_modelspc_data, ws, data=super_modelspc_data, persistent=False)

In [12]:
duplicates = set()
for mr in super_modelspc_data['modelreactions']:
    if not mr['id'] in duplicates:
        duplicates.add(mr['id'])
    else:
        mr['id'] += '_copy'
        duplicates.add(mr['id'])

In [13]:
morph.model = super_modelspc

In [15]:
res = morph.process_reactions2()

In [17]:
output_model_data = kbase.get_object(output_model_id, ws)
output_model_expected = cobrakbase.core.KBaseFBAModel(output_model_data)

In [18]:
output_model_actual = cobrakbase.core.KBaseFBAModel(new_model.get_object())

In [125]:
kbase.save_object("py3test_new", "filipeliu:narrative_1571685026282", "KBaseFBA.FBAModel", new_model.get_object())

[[6,
  'py3test_new',
  'KBaseFBA.FBAModel-11.0',
  '2019-10-21T19:51:14+0000',
  1,
  'filipeliu',
  50034,
  'filipeliu:narrative_1571685026282',
  '54ce37e5f95dbc542a90af858b0597b7',
  862467,
  {'Number gapgens': '0',
   'Type': 'SingleOrganism',
   'Number gapfills': '0',
   'Source ID': '2016_05_05_mmps',
   'Number biomasses': '1',
   'Number compartments': '2',
   'Genome': '48521/4/1',
   'Source': 'Imported',
   'Number compounds': '767',
   'Number reactions': '543',
   'Name': 'Methanocaldococcus jannaschii DSM 2661'}]]

In [182]:
#fbamodel = cobrakbase.core.KBaseFBAModel(model.get_object(False))
from cobrakbase.core.converters import KBaseFBAModelToCobraBuilder
m = cobrakbase.core.KBaseBiochemMedia(morph.media.get_object())
cobra_model = KBaseFBAModelToCobraBuilder(output_model_expected).with_media(m).build()
cobra_model.summary()

Infeasible: None (infeasible)

In [189]:
cobra_model.solver.write("/Users/fliu/Downloads/bad.lp")

AttributeError: 'Model' object has no attribute 'write'

In [192]:
lp = cobra_model.solver.to_lp()
with open('/Users/fliu/Downloads/bad.lp', 'w') as f:
    f.write(lp)

In [183]:
cplex = cobra.solvers.cplex_solver.create_problem(cobra_model)
cplex.write("/Users/fliu/Downloads/bad.lp")

AttributeError: module 'cobra' has no attribute 'solvers'

In [39]:
len(output_model_actual.reactions)

543

In [38]:
len(output_model_expected.reactions)

542

In [29]:
rxn_ids_actual   = set(map(lambda x : x.data['id'], output_model_actual.reactions))
rxn_ids_expected = set(map(lambda x : x.data['id'], output_model_expected.reactions))

In [31]:
both = (rxn_ids_actual & rxn_ids_expected)

In [32]:
len(both)

541

In [40]:
rxn_ids_actual - both

{'rxn10471_c0'}

In [42]:
growth_condition.evaluate({'morph': morph, 'model': FBAModel("", "", data=output_model_data, persistent=False)})

cobra/util/solver.py:416 UserWarning: solver status is 'infeasible'


False

In [161]:
import cobra
cobra.so

<module 'cobra.flux_analysis' from '/anaconda3/lib/python3.6/site-packages/cobra/flux_analysis/__init__.py'>

In [134]:

growth_condition.evaluate({'morph': morph, 'model': FBAModel("", "", data=new_model.get_object(), persistent=False)})

True

In [166]:
fba_data = kbase.get_object('fba', 'filipeliu:narrative_1571685026282')
fba_data['objectiveValue']

3.91244

In [127]:
solution = cobrakbase.core.KBaseFBA(fba_data)

In [63]:
total_sum = {}
for r in output_model_expected.reactions:
    flux = get_reaction_variable_by_id(solution, r.id)['value']
    #print(r, get_reaction_variable_by_id(solution, r.id)['value'])
    for m in r.metabolites:
        if not m.id in total_sum:
            total_sum[m.id] = 0
        total_sum[m.id] += r.metabolites[m] * flux

In [82]:
output_model_data.keys()

dict_keys(['__VERSION__', 'biomasses', 'delete_biomasses', 'deleted_reactions', 'gapfilledcandidates', 'gapfillings', 'gapgens', 'genome_ref', 'id', 'model_edits', 'modelcompartments', 'modelcompounds', 'modelreactions', 'name', 'quantopts', 'source', 'source_id', 'template_ref', 'template_refs', 'type'])

In [85]:
biomass['id']

'bio1'

In [157]:
total_sum = {}
for mr in output_model_data['modelreactions']:
    #print(mr)
    s_id = mr['id'].replace('-', '__DASH__')
    flux = growth_condition.fba.fluxes[s_id]
    #print(r, get_reaction_variable_by_id(solution, r.id)['value'])
    for o in mr['modelReactionReagents']:
        #print(o)
        if not o['modelcompound_ref'] in total_sum:
            total_sum[o['modelcompound_ref']] = 0
        total_sum[o['modelcompound_ref']] += o['coefficient'] * flux
        
for biomass in output_model_data['biomasses']:
    flux = growth_condition.fba.fluxes[biomass['id'] + '_biomass']
    #print(biomass)
    for biomasscompound in biomass['biomasscompounds']:
        coefficient = biomasscompound['coefficient']
        cpd = biomasscompound['modelcompound_ref']
        #print(coefficient, cpd, cpd in total_sum)
        if not cpd in total_sum:
            total_sum[cpd] = 0
        total_sum[cpd] += coefficient * flux

In [167]:
total_sum = {}
for mr in output_model_data['modelreactions']:
    #print(mr)
    flux = get_reaction_variable_by_id(solution, mr['id'])['value']
    #print(r, get_reaction_variable_by_id(solution, r.id)['value'])
    for o in mr['modelReactionReagents']:
        #print(o)
        if not o['modelcompound_ref'] in total_sum:
            total_sum[o['modelcompound_ref']] = 0
        total_sum[o['modelcompound_ref']] += o['coefficient'] * flux
        
for b in solution.data['FBABiomassVariables']:
    biomass_ref = b['biomass_ref'].split('/')[-1]
    flux = b['value']
    for biomass in output_model_data['biomasses']:
        if biomass['id'] == biomass_ref:
            #print(biomass)
            for biomasscompound in biomass['biomasscompounds']:
                coefficient = biomasscompound['coefficient']
                cpd = biomasscompound['modelcompound_ref']
                #print(coefficient, cpd, cpd in total_sum)
                if not cpd in total_sum:
                    total_sum[cpd] = 0
                total_sum[cpd] += coefficient * flux

In [168]:
import math
for m in total_sum:
    net = total_sum[m]
    if not net == 0:
        if m.endswith('_e0'):
            print(m, net)
cut = 0.0001
print('~~~~~~~~~~~~~~ internals >= ' + str(cut) + ' ~~~~~~~~~~~~~~')
for m in total_sum:
    net = total_sum[m]
    if not net == 0:
        if not m.endswith('_e0') and math.fabs(net) >= cut:
            print(m, net)
            
print('~~~~~~~~~~~~~~ internals  < ' + str(cut) + ' ~~~~~~~~~~~~~~')
for m in total_sum:
    net = total_sum[m]
    if not net == 0:
        if not m.endswith('_e0') and math.fabs(net) < cut:
            print(m, net)

~/modelcompounds/id/cpd00254_e0 -0.0124868
~/modelcompounds/id/cpd00205_e0 -0.011999999999943611
~/modelcompounds/id/cpd00067_e0 730.4699999999999
~/modelcompounds/id/cpd10515_e0 -0.0124868
~/modelcompounds/id/cpd00971_e0 -661.696
~/modelcompounds/id/cpd00149_e0 -0.0124868
~/modelcompounds/id/cpd00305_e0 -0.0124868
~/modelcompounds/id/cpd00073_e0 0.0249737
~/modelcompounds/id/cpd10516_e0 -0.0124868
~/modelcompounds/id/cpd00001_e0 180.707
~/modelcompounds/id/cpd00011_e0 -103.271
~/modelcompounds/id/cpd00063_e0 -0.0124868
~/modelcompounds/id/cpd00239_e0 -0.515361
~/modelcompounds/id/cpd11640_e0 -307.944
~/modelcompounds/id/cpd00099_e0 -0.0124868
~/modelcompounds/id/cpd00009_e0 2.38443
~/modelcompounds/id/cpd00013_e0 -15.7452
~~~~~~~~~~~~~~ internals >= 0.0001 ~~~~~~~~~~~~~~
~/modelcompounds/id/cpd00067_c0 -661.6984610200001
~/modelcompounds/id/cpd00001_c0 -0.0005220800001097814
~/modelcompounds/id/cpd00009_c0 -0.00015812000003734283
~/modelcompounds/id/cpd00047_c0 -0.0005132000000571679


In [120]:
for mr in output_model_data['modelreactions']:
    flux = get_reaction_variable_by_id(solution, mr['id'])['value']
    for o in mr['modelReactionReagents']:
        if 'cpd00971' in o['modelcompound_ref']:
            print(mr['id'], o['coefficient'], o['modelcompound_ref'], flux)

rxn05209_c0 1 ~/modelcompounds/id/cpd00971_e0 776.456
rxn05209_c0 -1 ~/modelcompounds/id/cpd00971_c0 776.456
Eha-Ehb_c0 -2 ~/modelcompounds/id/cpd00971_e0 995.357
Eha-Ehb_c0 2 ~/modelcompounds/id/cpd00971_c0 995.357
ATPS_c0 -4 ~/modelcompounds/id/cpd00971_e0 -303.564
ATPS_c0 4 ~/modelcompounds/id/cpd00971_c0 -303.564
rxn05209_c0 -1 ~/modelcompounds/id/cpd00971_e0 776.456
rxn05209_c0 1 ~/modelcompounds/id/cpd00971_c0 776.456


In [123]:
#cpd00971_e0 776.456 1
#cpd00971_e0 995.357 -2
#cpd00971_e0 -303.564 -4
#cpd00971_e0 776.456 -1
776.456 * 1 + 995.357 * -2 + -303.564 * -4

-0.0019999999997253326

In [52]:
def get_reaction_variable_by_id(f, rxn_id):
    for v in f.data['FBAReactionVariables']:
        v_id = v['modelreaction_ref'].split('/')[-1]
        if rxn_id == v_id:
            return v
    return None

In [169]:
%run ../../../scripts/bios_utils.py

In [170]:
fba_json = read_json('/Users/fliu/Downloads/fba_beta/9.json')

In [171]:
fba_json.keys()

dict_keys(['FBABiomassVariables', 'FBACompoundBounds', 'FBACompoundVariables', 'FBAConstraints', 'FBADeletionResults', 'FBAMetaboliteProductionResults', 'FBAMinimalMediaResults', 'FBAMinimalReactionsResults', 'FBAPromResults', 'FBAReactionBounds', 'FBAReactionVariables', 'FBATintleResults', 'MFALog', 'PROMKappa', 'QuantitativeOptimizationSolutions', '__VERSION__', 'additionalCpd_refs', 'allReversible', 'biomassRemovals', 'biomassflux_objterms', 'calculateReactionKnockoutSensitivity', 'comboDeletions', 'compoundflux_objterms', 'decomposeReversibleDrainFlux', 'decomposeReversibleFlux', 'defaultMaxDrainFlux', 'defaultMaxFlux', 'defaultMinDrainFlux', 'drainfluxUseVariables', 'fbamodel_ref', 'findMinimalMedia', 'fluxMinimization', 'fluxUseVariables', 'fva', 'gapfillingSolutions', 'geneKO_refs', 'id', 'inputfiles', 'maximizeActiveReactions', 'maximizeObjective', 'media_list_refs', 'media_ref', 'minimizeErrorThermodynamicConstraints', 'minimize_reaction_costs', 'minimize_reactions', 'noErrorT

In [176]:
lp = '\n'.join(fba_json['outputfiles']['LP_file'])

In [178]:
with open('/Users/fliu/Downloads/lp.lp', 'w') as f:
    f.write(lp)